In [20]:
import requests
import pandas as pd
from io import BytesIO
from zipfile import ZipFile

# US - San Diego data from "https://openpolicing.stanford.edu/data/"
file_url = "https://stacks.stanford.edu/file/druid:yg821jf8611/yg821jf8611_ca_san_diego_2020_04_01.csv.zip"

resp = requests.get(file_url) # Downloading the dataset
zip_file = ZipFile(BytesIO(resp.content)) # Loading the ZipFile into memory
csv_file_name = [n for n in zip_file.namelist() if n.endswith(".csv")][0] # Taking the csv filename

## Inspecting and cleaning the data

In [21]:
df: pd.DataFrame = pd.read_csv(zip_file.open(csv_file_name))

print(df.shape)
df.head(3)


(383027, 21)


,raw_row_number,date,time,service_area,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,outcome,contraband_found,search_conducted,search_person,search_vehicle,search_basis,reason_for_search,reason_for_stop,raw_action_taken,raw_subject_race_description
0,1,2014-01-01,01:25:00,110,24.0,white,male,vehicular,False,True,...,citation,NaN,False,False,False,NaN,NaN,Moving Violation,Citation,WHITE
1,2,2014-01-01,05:47:00,320,42.0,white,male,vehicular,False,False,...,warning,NaN,False,False,False,NaN,NaN,Moving Violation,Verbal Warning,WHITE
2,3,2014-01-01,07:46:00,320,29.0,asian/pacific islander,male,vehicular,False,False,...,warning,NaN,False,False,False,NaN,NaN,Moving Violation,Verbal Warning,LAOTIAN


In [15]:
df.isna().sum()

raw_row_number                       0
date                               183
time                               735
service_area                         0
subject_age                      11963
subject_race                      1234
subject_sex                        661
type                                 0
arrest_made                      34743
citation_issued                  31971
warning_issued                   31971
outcome                          39172
contraband_found                366739
search_conducted                     0
search_person                     2190
search_vehicle                    2190
search_basis                    366739
reason_for_search               368749
reason_for_stop                    219
raw_action_taken                 31971
raw_subject_race_description      1234
dtype: int64

In [22]:
# Dropping columns that has a lot of missing values
df.drop(["contraband_found", "search_basis", "reason_for_search"], axis=1, inplace=True)
df.shape

(383027, 18)

In [23]:
# Dropping the rows that has missing values in any of the field
df.dropna(axis=0, inplace=True)
df.shape

(312998, 18)

In [32]:
# combining 'date' and 'time' column to 'date_time'
combine = df.date.str.cat(df.time, sep=' ')
df["date_time"] = pd.to_datetime(combine)
 
print(df.shape)
sum(df.isna().sum())

(312998, 19)


0

## Analyzing data

In [37]:
df.outcome.value_counts(normalize=True)

citation    0.639643
warning     0.348612
arrest      0.011744
Name: outcome, dtype: float64

In [42]:
df.groupby("raw_subject_race_description")["outcome"].value_counts(normalize=True)

raw_subject_race_description  outcome 
ASIAN INDIAN                  citation    0.705882
                              warning     0.294118
BLACK                         citation    0.537975
                              warning     0.442443
                              arrest      0.019582
CAMBODIAN                     warning     0.496689
                              citation    0.463576
                              arrest      0.039735
CHINESE                       citation    0.595789
                              warning     0.401053
                              arrest      0.003158
FILIPINO                      citation    0.569369
                              warning     0.419926
                              arrest      0.010706
GUAMANIAN                     citation    0.585366
                              warning     0.317073
                              arrest      0.097561
HAWAIIAN                      citation    0.580000
                              warning     0